### scraping poder360 - site de noticias

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv
import json

load_dotenv()

def get_links(url):
    # Enviando requisição HTTP para obter o conteúdo da página
    response = requests.get(url)
    content = response.content

    # Analisando o conteúdo da página com BeautifulSoup
    soup = BeautifulSoup(content, 'html.parser')

    # Encontrar todas as tags ul que tenham archive-list__list na classe
    archive_list = soup.find('ul', class_='archive-list__list')

    # Encontrar todas as tags <li> dentro da tag <ul>
    li_tags = archive_list.find_all('li')

    links = []
    # Iterar sobre todas as tags <li> e extrair o href
    for li in li_tags:
        a_tag = li.find('a')
        href = a_tag['href']
        links.append(href)
    return links

def extract_article_info(url):
    try:
        # Fazer a requisição HTTP
        response = requests.get(url)
        
        # Verificar se a requisição foi bem-sucedida
        if response.status_code != 200:
            return f"Erro ao acessar a página: {response.status_code}"
            
        # Definir a codificação como UTF-8
        response.encoding = 'utf-8'

        # Fazer o parsing do HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extrair informações do artigo
        if url.startswith('https://www.poder360.com.br/poder-flash'):
            title = soup.find('h1', class_='box-poder-flash__title mt-4').text.strip()
        else:
            title = soup.find('h1', class_='inner-page-section__title title-1').text.strip()
        subtitle = soup.find('h2', class_='inner-page-section__line').text.strip()
        
        # Tentar encontrar a data e o autor
        date = soup.find('time')  # Assumindo que a data está dentro de uma tag <time>
        if date:
            date = date.text.strip()
        else:
            date = "Data não encontrada"
        
        # Extrair o autor
        author_div = soup.find('div', class_='footer-post__box')
        if author_div:
            author_name = author_div.find('a', class_='author__name')
            if author_name:
                author = author_name.text.strip()
            else:
                author = "Autor não encontrado"
        else:
            author = "Autor não encontrado"
        
        # Extrair o texto do artigo
        article_body = soup.find('div', class_='inner-page-section__text')
        if article_body:
            # Encontra e remove o elemento <form> dentro de article_body se ele existir
            form = article_body.find('form')
            if form:
                form.decompose()

            # Inicializa uma lista para armazenar o texto dos elementos
            text_elements = []

            # Itera sobre todos os filhos do article_body
            for child in article_body.children:
                if child.name == 'p':
                    text_elements.append(child.get_text(strip=True))
                elif child.name == 'ul':
                    # Itera sobre todos os <li> dentro do <ul>
                    for li in child.find_all('li'):
                        text_elements.append(li.get_text(strip=True))

            # Combina todos os textos em uma única string separada por quebras de linha
            text = '\n'.join(text_elements)
        else:
            text = "Texto do artigo não encontrado"
        
        # Concatenar informações em uma string
        article_info = f"titulo: {title}\nsubtitulo: {subtitle}\ndata: {date}\nautor: {author}\n\ntexto: {text}\n\n"
        
        return article_info
    except Exception as e:
        print(f"Erro ao extrair informações do artigo: {e}")
        print(f"URL do artigo: {url}")

def my_scheduled_task(n_dias):
    # Calcular a data de ontem
    for i in range(1,n_dias):
        yesterday = datetime.now() - timedelta(i)
        yesterday_str = yesterday.strftime('%Y-%m-%d')

        #construir a url das notícias de ontem
        url = f'https://www.poder360.com.br/{yesterday.strftime("%Y/%m/%d")}'
        print(f"Date link: {url}")

        links = get_links(url)
        links_filtrados = [link for link in links if not link.startswith("https://www.poder360.com.br/author/")]
        print(f"Cron job links: {len(links_filtrados)}")

        news = []
        for link in links_filtrados:
            article_info = extract_article_info(link)
            news.append(article_info)
        
        # Nome do arquivo
        filename = f'news_{yesterday_str}.json'

        # Tentar abrir o arquivo e ler seu conteúdo, se existir
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                data = json.load(f)
        except FileNotFoundError:
            data = []

        # Adicionar a nova notícia à lista
        data.append(news)

        # Salvar a lista de notícias no arquivo JSON
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)


        print("Cron job finished")
    

if __name__ == "__main__":
    my_scheduled_task(15)

Date link: https://www.poder360.com.br/2024/09/02
Cron job links: 119
Cron job finished
Date link: https://www.poder360.com.br/2024/09/01
Cron job links: 56
Cron job finished
Date link: https://www.poder360.com.br/2024/08/31
Cron job links: 76
Cron job finished
Date link: https://www.poder360.com.br/2024/08/30
Cron job links: 142
Cron job finished
Date link: https://www.poder360.com.br/2024/08/29
Cron job links: 117
Cron job finished
Date link: https://www.poder360.com.br/2024/08/28
Cron job links: 149
Cron job finished
Date link: https://www.poder360.com.br/2024/08/27
Cron job links: 148
Cron job finished
Date link: https://www.poder360.com.br/2024/08/26
Cron job links: 128
Cron job finished
Date link: https://www.poder360.com.br/2024/08/25
Cron job links: 61
Cron job finished
Date link: https://www.poder360.com.br/2024/08/24
Cron job links: 83
Cron job finished
Date link: https://www.poder360.com.br/2024/08/23
Cron job links: 84
Cron job finished
Date link: https://www.poder360.com.b

In [4]:
# criar um arquivo json com todas as noticias

for json_file in os.listdir():
    if json_file.startswith('news_'):
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        try:
            with open('news.json', 'r', encoding='utf-8') as f:
                all_data = json.load(f)
        except FileNotFoundError:
            all_data = []
        
        all_data.extend(data)
        
        with open('news.json', 'w', encoding='utf-8') as f:
            json.dump(all_data, f, ensure_ascii=False, indent=4)
            
        os.remove(json_file)
        


In [1]:
# ler dados 
import pandas as pd
import json


with open('news.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
    
flattened_data = [item for sublist in data for item in sublist]


# df = pd.DataFrame(flattened_data, columns=["content"])


In [4]:
def parse_article(article):
    fields = ["titulo", "subtitulo", "data", "autor", "texto"]
    parsed_data = {}

    for field in fields:
        if f"{field}:" in article:
            # Encontrar o início do campo
            start = article.index(f"{field}:") + len(f"{field}:")
            # Encontrar o fim do campo (ou fim do texto)
            end = len(article)
            for next_field in fields:
                if next_field != field and f"{next_field}:" in article[start:]:
                    potential_end = article.index(f"{next_field}:", start)
                    if potential_end < end:
                        end = potential_end
            # Extrair o valor do campo
            parsed_data[field] = article[start:end].strip()

    return parsed_data

# Transformar a lista de artigos em uma lista de dicionários
parsed_data = [parse_article(article) for article in flattened_data]

# Converter para DataFrame
df = pd.DataFrame(parsed_data)

In [6]:
df.head()

,titulo,subtitulo,data,autor,texto
0,Barroso lança livro sobre Inteligência Artific...,Presidente do STF fez sessão de autógrafos na ...,20.ago.2024 (terça-feira) - 23h28,PODER360,"O presidente doSTF(Supremo Tribunal Federal), ..."
1,Marçal apaga 4º vídeo com críticas a Boulos ap...,"Segundo o TRE de São Paulo, o conteúdo não tem...",20.ago.2024 (terça-feira) - 23h21,Autor não encontrado,O candidato à Prefeitura de São PauloPablo Mar...
2,Kamala aceita de forma simbólica nomeação do P...,Democrata diz estar “honrada” por receber indi...,20.ago.2024 (terça-feira) - 23h08,PODER360,"A vice-presidente dos Estados Unidos,Kamala Ha..."
3,Janja participa da cerimônia de Restauração do...,Primeira-dama foi convidada pelo ministro da J...,20.ago.2024 (terça-feira) - 23h01,PODER360,"A primeira-dama, Janja Lula da Silva, particip..."
4,"Trump não tem empatia moral, diz ex-secretária...",Ex-apoiadores discursam no 2º dia da Convenção...,20.ago.2024 (terça-feira) - 22h05,PODER360,A ex-secretária de imprensa da Casa Branca do ...


In [7]:
# salvar em csv
df.to_csv("news.csv", index=False)